# Обработка текстов на естественном языке: задача классификации

## Создание набора данных

### Обзор

Этот блокнот нужен для создания необработанного датасета для задачи классификации предложений по признаку авторства из оригинальных текстов. На выходе будет csv файл в формате: **"Предложение", "автор"**. 

Для создания датасета используются произведения: 


|Автор|Произведение|
|---------  |-------|
|А.П. Чехов | Сборник рассказов и повестей |
|Ф.М. Достоевский| Сборник избранных сочинений |
|Л.Н. Толстой| Самые популярные сочинения |

Из репозитория https://github.com/d0rj/RusLit, в котором собраны тексты русских авторов из открытых источников.

### Создание файла

#### Импорт пакетов

In [1]:
import glob
from nltk import tokenize, download
import numpy as np
import random
import pandas as pd

from typing import List

Необходимо для токенизации предложения, достаточно вызвать на рабочей машине один раз

In [2]:
download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dimab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Функция для загрузки и предобработки текста

Создадим список предложений, длина которых более 5ти символов, так как более короткие, скорее всего, не несут полезной для определения авторства информации. Вообще говоря, эти предложения могут выражать, и выражать довольно ярко, стиль письма конкретного автора; однако в модели это не используется.

Для улучшения работы токенизатора предложений проводятся замены некоторых комбинаций символов. Так, реплики будут отдельными от  речи автора предложениями, а проблема с кавычками должна быть решена.

In [3]:
def split_text(filepath: str, min_char: int = 5) -> List[str]:
    
    text = str()
    with open(filepath, 'r', encoding='utf8') as file:
        text = file.read().replace('\n', '. ')
        text = text.replace('.”', '”.').replace('."', '".').replace('?”', '”?').replace('!”', '”!')
        text = text.replace('--', ' ').replace('. . .', '').replace('_', '')
    
    sentences = tokenize.sent_tokenize(text)    
    sentences = [sentence for sentence in sentences if len(sentence) >= min_char]

    return list(sentences)

#### Создание набора предложений для каждого автора

In [4]:
chekhov = []
for path in glob.glob('./input/Chekhov/*.txt'):
    chekhov += split_text(path)
    
dostoevsky = []
for path in glob.glob('./input/Dostoevsky/*.txt'):
    dostoevsky += split_text(path)

tolstoy = []
for path in glob.glob('./input/Tolstoy/*.txt'):
    tolstoy += split_text(path)

In [5]:
text_dict = { 'Чехов': chekhov, 'Достоевский': dostoevsky, 'Толстой': tolstoy }

for key in text_dict.keys():
    print(key, ':', len(text_dict[key]), ' предложений')

Чехов : 21860  предложений
Достоевский : 77817  предложений
Толстой : 41769  предложений


Каждый список содержит от 20535 до 77817 предложений. Чтобы в нашем наборе было равномерное распределение авторов, ограничим набор для каждого, к примеру, 18000 предложениями.

#### Комбинируем предложения

In [6]:
np.random.seed(1)

max_len = 18_000

names = [chekhov, dostoevsky, tolstoy]

combined = []
for name in names:
    name = np.random.choice(name, max_len, replace = False)
    combined += list(name)

print('Длина комбинированного и внутренне перемешанного списка:', len(combined))

Длина комбинированного и внутренне перемешанного списка: 54000


#### Создаём маркированный список

В этом моменте важно указывать метки авторов (их имена) в том же порядке, что и на предыдущем шаге, иначе данные просто окажутся неверными. Пока что простой регулирующий это механизм в голову не приходит. 

In [7]:
labels = ['Чехов'] * max_len + ['Достоевский'] * max_len + ['Толстой'] * max_len

print('Длина маркированного списка:', len(labels))

Длина маркированного списка: 54000


Вывод количества был нужен для дополнительного контроля за данными и их метками. Равенство говорит о том, что каждое предложение в нашем наборе данных будет иметь метку (правильную или не правильную - контролироваться должно было раньше). 

In [8]:
len(combined) == len(labels)

True

#### Случайно перемешиваем данные

In [9]:
random.seed(3)

zipped = list(zip(combined, labels))
random.shuffle(zipped)
combined, labels = zip(*zipped)

**Экспорт получившегося набора данных**

In [10]:
out_data = pd.DataFrame()
out_data['text'] = combined
out_data['author'] = labels

In [11]:
print(out_data.head())
print(out_data.tail())

                                                text       author
0  Вы не знаете, как вы для меня важны и как вы м...      Толстой
1  14)..    Однако эта часть мемуаров не была про...      Толстой
2  Он прибежал из леса к опушке и, бледный, с рас...        Чехов
3  — сказала мать, притворно сердито отталкивая д...      Толстой
4  Вечер проектировался, однако же, запросто; ожи...  Достоевский
                                                    text       author
53995  Хвалить войну никто не решится.. – Но вы говор...  Достоевский
53996                     Ты вот так и норовишь уколоть!        Чехов
53997  – Давеча Гаврила Ардалионович Ивану Федоровичу...  Достоевский
53998                  Армия не могла нигде поправиться.      Толстой
53999  Минут через пять она еще раз повернулась, закр...        Чехов


In [12]:
out_data.to_csv('author_data.csv', index=False)